In [114]:
# import packages

# import clevercsv as csv
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import csv
%matplotlib inline

In [115]:
plt.style.use('dark_background')
pio.templates.default = "plotly_dark"

In [116]:
col_names= ['JOBID', 'ACCOUNT', 'USER', 'REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME',
       'NODES', 'CPUS', 'PARTITION', 'EXITCODE', 'STATE', 'NODE1','NODE2','NODE3','NODE4','NODE5','NODE6']
accre_df = pd.read_csv('accre-jobs-2020.csv', names=col_names) 


C:\Users\matth\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,8,13,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [117]:
accre_df.shape

(3816291, 18)

In [118]:
rslt_df = accre_df[accre_df['CPUS'] == 'CPUS']

In [119]:
rslt_df

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODE1,NODE2,NODE3,NODE4,NODE5,NODE6
0,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,NaN,NaN,NaN,NaN,NaN


In [120]:
accre_df = accre_df[1:]

In [121]:
accre_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODE1,NODE2,NODE3,NODE4,NODE5,NODE6
1,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,NaN,NaN,NaN,NaN,NaN
2,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,NaN,NaN,NaN,NaN,NaN
3,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,NaN,NaN,NaN,NaN,NaN
4,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,NaN,NaN,NaN,NaN,NaN
5,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,NaN,NaN,NaN,NaN,NaN


In [122]:
accre_df['CPUS']= accre_df['CPUS'].astype(str).astype(float)

In [123]:
accre_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODE1,NODE2,NODE3,NODE4,NODE5,NODE6
1,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24.0,production,0:0,COMPLETED,cn1531,NaN,NaN,NaN,NaN,NaN
2,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24.0,production,0:0,COMPLETED,cn1441,NaN,NaN,NaN,NaN,NaN
3,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24.0,production,0:0,COMPLETED,cn1464,NaN,NaN,NaN,NaN,NaN
4,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24.0,production,0:0,COMPLETED,cn1473,NaN,NaN,NaN,NaN,NaN
5,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24.0,production,0:0,COMPLETED,cn1440,NaN,NaN,NaN,NaN,NaN


In [124]:
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 1 to 3816290
Data columns (total 18 columns):
 #   Column     Dtype  
---  ------     -----  
 0   JOBID      object 
 1   ACCOUNT    object 
 2   USER       object 
 3   REQMEM     object 
 4   USEDMEM    object 
 5   REQTIME    object 
 6   USEDTIME   object 
 7   NODES      object 
 8   CPUS       float64
 9   PARTITION  object 
 10  EXITCODE   object 
 11  STATE      object 
 12  NODE1      object 
 13  NODE2      object 
 14  NODE3      object 
 15  NODE4      object 
 16  NODE5      object 
 17  NODE6      object 
dtypes: float64(1), object(17)
memory usage: 524.1+ MB


In [125]:
accre_df['CPUS'].unique

<bound method Series.unique of 1          24.0
2          24.0
3          24.0
4          24.0
5          24.0
           ... 
3816286     2.0
3816287     2.0
3816288     2.0
3816289     8.0
3816290     1.0
Name: CPUS, Length: 3816290, dtype: float64>

In [130]:
#new column is for requested memory per node
accre_df['RMPN'] = accre_df['REQMEM'].str.extract('(.*)Mn$')
accre_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODE1,NODE2,NODE3,NODE4,NODE5,NODE6,RMPN
1,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24.0,production,0:0,COMPLETED,cn1531,NaN,NaN,NaN,NaN,NaN,122880
2,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24.0,production,0:0,COMPLETED,cn1441,NaN,NaN,NaN,NaN,NaN,122880
3,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24.0,production,0:0,COMPLETED,cn1464,NaN,NaN,NaN,NaN,NaN,122880
4,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24.0,production,0:0,COMPLETED,cn1473,NaN,NaN,NaN,NaN,NaN,122880
5,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24.0,production,0:0,COMPLETED,cn1440,NaN,NaN,NaN,NaN,NaN,122880


In [126]:
accre_df['REQMEMNODE'] = accre_df['REQMEM'].str.extract('.*n$')
accre_df

ValueError: pattern contains no capture groups

In [80]:
accre_df.groupby(['CPUS']).size().sort_values(ascending=True)
accre_df

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODE1,NODE2,NODE3,NODE4,NODE5,NODE6
1,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24.0,production,0:0,COMPLETED,cn1531,NaN,NaN,NaN,NaN,NaN
2,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24.0,production,0:0,COMPLETED,cn1441,NaN,NaN,NaN,NaN,NaN
3,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24.0,production,0:0,COMPLETED,cn1464,NaN,NaN,NaN,NaN,NaN
4,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24.0,production,0:0,COMPLETED,cn1473,NaN,NaN,NaN,NaN,NaN
5,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24.0,production,0:0,COMPLETED,cn1440,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816286,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2.0,production,0:0,COMPLETED,cn312,NaN,NaN,NaN,NaN,NaN
3816287,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2.0,production,0:0,COMPLETED,cn312,NaN,NaN,NaN,NaN,NaN
3816288,24173815_10,portabella,vennie,32768Mn,21207.96M,1-00:00:00,00:01:40,1,2.0,production,0:0,COMPLETED,cn368,NaN,NaN,NaN,NaN,NaN
3816289,24173817,boysenberries,hortensia,100000Mn,0,1-00:00:00,00:03:05,1,8.0,production,0:0,RUNNING,cn1509,NaN,NaN,NaN,NaN,NaN


In [127]:
#testing
d = {'col1': ['29348Mn','239039Mc']}
df = pd.DataFrame(data=d)
df

,col1
0,29348Mn
1,239039Mc


In [108]:
x = '29348Mn'
re.findall('.*Mn$',x)

['29348Mn']

In [128]:
df['new'] = df['col1'].str.extract('(.*)Mn$')

In [71]:
accre_df['NODES'].unique()

array(['NODES', '1', '4', '2', '3', 1, 5, 38, 30, 25, 21, 13, 19, 23, 2,
       3, 4, 12, 51, 37, 75, 126, 11, 10, 20, 40, 41, 97, 45, 6, 63, 8,
       143, 15, 24, 35, 48, 80, 99, 120, 64, 121, 39, 55, 9, 66, 27, 17,
       28, 53, 34, 70, 67, 59, 77, 58, 43, 31, 61, 52, 50, 16, 26, 22, 14,
       112, 7, 95, 109, 100, 149, 146, 114, 94, 87, 140, 142, 44, 156,
       163, 62, 129, 68, 200, 400, 500, 18, 93], dtype=object)

In [72]:
accre_df['CPUS'].unique()

array(['CPUS', '24', '1', '16', '7', '4', '12', '2', '5', '8', '11', '6',
       '3', '9', '32', '64', '0', '10', 1, 12, 16, 8, 32, 4, 10, 2, 3, 5,
       7, 6, 11, 64, 9, 20, 24, 15, 14, 13, 18, 22, 0, 19], dtype=object)

In [ ]:
#split
np.where 
df[] = endwith

In [78]:
# split where ends with c and m

In [ ]:
# replace mn and mc with nothing

In [ ]:
#aggreaget columns


In [77]:
result = accre_df['REQMEM'].endswith('Mc')
result 

AttributeError: 'Series' object has no attribute 'endswith'

In [76]:
accre_df['NODES']= accre_df['NODES'].astype(str).astype(float)

ValueError: could not convert string to float: 'NODES'

In [75]:
accre_df['CPUS']= accre_df['CPUS'].astype(str).astype(float)

ValueError: could not convert string to float: 'CPUS'

In [73]:
accre_df = ['NODES'].astype(str).str.isdigit()

AttributeError: 'list' object has no attribute 'astype'

In [ ]:
accre_df = ['CPUS'].astype(str).str.isdigit()

In [ ]:
clean_accre_df.groupby('ACCOUNT').mean()

In [60]:
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816291 entries, 0 to 3816290
Data columns (total 18 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   ACCOUNT    object
 2   USER       object
 3   REQMEM     object
 4   USEDMEM    object
 5   REQTIME    object
 6   USEDTIME   object
 7   NODES      object
 8   CPUS       object
 9   PARTITION  object
 10  EXITCODE   object
 11  STATE      object
 12  NODE1      object
 13  NODE2      object
 14  NODE3      object
 15  NODE4      object
 16  NODE5      object
 17  NODE6      object
dtypes: object(18)
memory usage: 524.1+ MB


In [62]:
clean_accre_df = accre_df.drop(["JOBID","USER","REQTIME",
                                "USEDTIME","NODES","EXITCODE",
                               "NODE1","NODE2","NODE3","NODE4","NODE5","NODE6"], axis =1) # axis = 1 so that it works across our columns
clean_accre_df.head()

,ACCOUNT,REQMEM,USEDMEM,CPUS,PARTITION,STATE
0,ACCOUNT,REQMEM,USEDMEM,CPUS,PARTITION,STATE
1,treviso,122880Mn,65973.49M,24,production,COMPLETED
2,treviso,122880Mn,67181.12M,24,production,COMPLETED
3,treviso,122880Mn,69111.86M,24,production,COMPLETED
4,treviso,122880Mn,65317.33M,24,production,COMPLETED


In [64]:
x = "122880Mn"
x.replace("Mn","")

'122880'

In [63]:
clean_accre_df['REQMEM'] = accre_df['REQMEM'].replace("Mn","")
clean_accre_df['REQMEM'].head()

0      REQMEM
1    122880Mn
2    122880Mn
3    122880Mn
4    122880Mn
Name: REQMEM, dtype: object

In [65]:
clean_accre_df.groupby('ACCOUNT').mean()

DataError: No numeric types to aggregate

In [32]:
accre_df["ACCOUNT"].value_counts()

cms          807408
cep          609278
summer       380788
galia        198180
carrot       192524
              ...  
purple            1
discovery         1
crowns            1
hass              1
flamingo          1
Name: ACCOUNT, Length: 173, dtype: int64

In [10]:
accre_df['ACCOUNT'].unique()

array(['treviso', 'mickey', 'fava', 'delblush', 'fresh', 'minnewashta',
       'broccoli', 'mangosteen', 'pepino', 'sprite', 'mixed',
       'horseradish', 'galia', 'cepuna', 'conference', 'cantared',
       'harovin', 'turban', 'lychees', 'cep', 'seeded', 'sunburst',
       'wood', 'regal', 'crab', 'blackberries', 'portabella', 'gobo',
       'salustiana', 'cms', 'pasilla', 'cms_lowprio', 'winged', 'bulk',
       'summer', 'baking', 'sharlin', 'carrot', 'endive', 'cmsadmin',
       'peanuts', 'morita', 'water', 'muscat', 'boysenberries', 'concord',
       'bunch', 'cactus', 'large', 'orange', 'anise', 'bon', 'brazilnuts',
       'retailer', 'rosemarie', 'alkmene', 'bibb', 'pioppino', 'dill',
       'clara', 'sweet', 'casaba', 'chickpeas', 'cubanelle',
       'blueberries', 'arra', 'bell', 'enoki', 'carambola', 'pinole',
       'belle', 'fiddlehead', 'long', 'round', 'sunflower', 'grisette',
       'lollo', 'st', 'lemon', 'basil', 'fairchild', 'baby', 'sturmer',
       'texas', 'celery

In [23]:
accre_df['CPUS'].unique()

array([24,  1, 16,  7,  4, 12,  2,  5,  8, 11,  6,  3,  9, 32, 64,  0, 10,
       20, 15, 14, 13, 18, 22, 19], dtype=int64)

In [ ]:
# remove letters and convert to int for memory usage

In [ ]:
# convert mn to mc

In [14]:
# convert memory to per core


In [17]:
# create columns for time and percent comparison 

In [ ]:
# find average percent for each group

In [54]:
x = "122880Mn"
x.replace("Mn","")
#re.findall('\[^a-z0-90]/g', x)

'122880'

In [13]:
# groups are account
accre_df.head(10)
# need account, reqmem,usedmem, nodes, cpus, production

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473


# Question 3
What groups are best optimizing their memory usage in terms of percent of actual memory used of the memory requested for a job? What is the average percent for each group?

# Answer 1:
x is the best group for using 

# Answer 2:
The average percent for each group is